Свойкин Е.В. 2021 г.

## Приток к протяженной вертикальной трещине при псевдоустановившемся режиме

При описании установившегося течения флюида предполагалось выполнение закона сохранения расхода через любой элемент пласта. В реальности же приток вдоль каждой линии тока постоянно увеличивается. Чтобы учесть этот факт в теории, проще всего сделать допущение, что режим течения псевдоустановившийся. При этом необходимо сделать ряд дополнительных предположений: 
* сжимаемость флюида постоянная;
* после окончания начального переходного этапа добычи темп падения давления в каждой точке пласта постоянен, при этом объем нефти вследствие ее расширения увеличивается суммарный объем притока к добывающим скважинам. 

Положим, что границы пласта непроницаемы, следовательно, можно там считать приток нулевым. Учитывая все эти предположения запишем уравнение:

$$ k_x \frac{∂^{2}Φ}{∂x^{2}} +  k_y \frac{∂^{2}Φ}{∂y^{2}} + k_z \frac{∂^{2}Φ}{∂z^{2}} = - \frac{qμ}{Ah} $$

где $q$ - дебит, получаемый из пласта обхемом $Ah$.

Рассмотрим случай линейного псевдоустановившегося притока к вертикальной трещине, расположенной в центре пласта шириной 2$W$. По мере продвижения к трещине приток к обоим ее концам увеличивается по линейной зависимости. Если предположить, что расход определяется из уравнения Дарси, получим следующее соотношение:
 
$$ q_z = \frac{kLh}{μ} \frac{dP}{dx} = \frac{q}{2}\left( 1 - \frac{|x|}{W}\right) $$

Проинтегрировав которое по $x$ (с учетом граничного условия га поверхности трещины $ P = P_w $), будем иметь:

$$ P = P_w + \frac{qμ}{2kLh}\left( |x| - \frac{x^{2}}{2W}\right) $$

Полученное уравнение отличается от аналогичного уравнение для установившегося притока наличием дополнительного слагаемого. В призабойной зоне скважины наличие этого члена мало влияет на значение давление, а вот по мере удаления от трещины его влияние возрастает: чем ближе к границе пласта, тем меньше градиент давления (на границе снижается до нуля).

Среднее по объему пластовое давление $ P_a $ получается из следующего уравнения:

$$ P_a = P_w + \frac{qμW}{6kLh} $$ 

Разрешая это уравнение относительно $q$, получаем:

$$ q = \frac{6kLh\left( P_a - P_w\right)}{μW} $$
*(для каждой из сторон)

ref "Horizontal Wells for the Recovery of Oil, Gas and Bitumen by Roger M. Butler, p. 146"

# Построим модель в opm

In [1]:
from model_create import ModelGenerator, clear_folders
clear_folders()
# Параметры, которые необходимо расчитать:
parameters = ['FOPT', 'WOPR:*', 'FPR', 'WBHP:P1',]# 'FWPT', 'FLPT', 'FGPT', 'FWIT']

# Зададим дату начала расчета и продолжительность:
start_date = u"1 'JAN' 2020"
mounths = 5 # количество итераций;
days = 60 # продолжительность итерации;

# Зададим размеры модели:
lgr = True # локальное измельчение сетки по центру;
cells_cy = 20 # количество измельченных ячеек по y;
cells_cx = 4 # количество измельченных ячеек по x;
cells_v = 10 # размер измельченных ячеек, м;
nx = 40 # количество ячеек по х;
ny = 40 # количество ячеек по y;
nz = 10 # количество ячеек по z;
lx = 1500 # длина модели по x, м;
ly = lx # длина модели по y, м;
dz = 1 # размер ячейки по z, м;
tops_depth = 2500 # глубина залегания пласта, м;

# Зададим пористость и проницаемость:
por = 0.3 # пористость, д.ед;

# Зададим название, расположение добываюещей скважины и ее режим работы:
horizontal = [True] # Идентификатор горизонтальной скважины для скрипта
prod_names = ['P1'] # название скважин;
prod_xs=[20] # координата скважин по x;
prod_ys=[11] # координата скважин по y (если horizontal=true - начальная координата);
y_stop = [31] # координата конца горизонтальной скважины, если horizontal=true;
prod_z1s=[1] # начало интервала вскрытия по z;
prod_z2s=[6] # конец интервала вскрытия по z (если horizontal=true, то показывает уровень вскрытия);
rezim = ['BHP'] # Режим работы скважины
prod_q_oil = ['*'] # дебит, м3/сут;
prod_bhp = ['200'] # забойное давление, атм;

# Другие настройки модели:
only_prod = True # Модель только с добывающими скважинами
upr_rezim_water = False # Моделируем упруго-водонапорный режим
upr_rezim_gas = False # Моделируем газонапорный режим
neogr_plast = True # Моделируем неограниченный пласт

# Задаем скин, радиус скважины (последовательно для добывающих и нагнетательных скважин):
skin = [0]
rw = [0.073]

# Свойства продукции:
oil_den = 860 # плотность нефти, кг/м3;
wat_den = 1010 # плотность воды, кг/м3;
gas_den = 0.9 # плотность газа, кг/м3;

# Свойства поороды:
rock_compr = 1.5E-005 # сжимаемость породы, Па^-1

# EQUILIBRIUM DATA:
p_depth = 2500 # Глубина замера пластового давления, м;
p_init = 320  # Начальное пластовое давление, атм;
o_w_contact = 2550 # Глубина ВНК, м;
pc_woc = 0 # Капиллярное давление на ВНК 
g_o_contact = 2400 # Глубина ГНК, м;
pc_goc = 0 # Капиллярное давление на ГНК

In [2]:
#выбираем шаблон data файла для различных симуляторов
template = 1 # : 1-opm; 2-ecl (в разработке)
permx_list = [5, 50, 200] # проницаемость по x, мД;
kk = [0.25, 0.5, 1, 3]
model_list = []
for j in range(0, 4):
    for i in range(0, 3):
        permx = permx_list[i]
        permy = permx_list[i] # проницаемость по y, мД;
        permz = permx_list[i]*kk[j] # проницаемость по z, мД;
        model_name = f'TEST_MODEL_HORIZONTAL.{j}{i}'
        result_name = model_name
        model_list.append(' горизонтальная проницаемость - ' + str(permx) + ' мД')
        model = ModelGenerator(start_date=start_date, mounths=mounths,
                        days=days, nx=nx, ny=ny, nz=nz, dz=dz, por=por, permx=permx,
                        permy=permy, permz=permz, prod_names=prod_names, prod_xs=prod_xs,
                        prod_ys=prod_ys, prod_z1s=prod_z1s, prod_z2s=prod_z2s, prod_q_oil=prod_q_oil,
                        skin=skin, oil_den=oil_den, wat_den=wat_den, gas_den=gas_den, p_depth=p_depth, 
                        p_init=p_init, o_w_contact=o_w_contact, pc_woc=pc_woc, g_o_contact=g_o_contact,
                        pc_goc=pc_goc, tops_depth = tops_depth,
                        rezim=rezim, prod_bhp=prod_bhp, horizontal=horizontal, y_stop=y_stop, only_prod=only_prod,
                        lgr=lgr, lx=lx, ly=ly, cells_cy=cells_cy, cells_v=cells_v, cells_cx=cells_cx,
                        upr_rezim_water=upr_rezim_water, upr_rezim_gas=upr_rezim_gas, rw=rw, template=template, neogr=neogr_plast)
        model.create_model(model_name, result_name, parameters)

TEST_MODEL_HORIZONTAL.00_RESULT.csv is created
TEST_MODEL_HORIZONTAL.00.csv is read
TEST_MODEL_HORIZONTAL.01_RESULT.csv is created
TEST_MODEL_HORIZONTAL.01.csv is read
TEST_MODEL_HORIZONTAL.02_RESULT.csv is created
TEST_MODEL_HORIZONTAL.02.csv is read
TEST_MODEL_HORIZONTAL.10_RESULT.csv is created
TEST_MODEL_HORIZONTAL.10.csv is read
TEST_MODEL_HORIZONTAL.11_RESULT.csv is created
TEST_MODEL_HORIZONTAL.11.csv is read
TEST_MODEL_HORIZONTAL.12_RESULT.csv is created
TEST_MODEL_HORIZONTAL.12.csv is read
TEST_MODEL_HORIZONTAL.20_RESULT.csv is created
TEST_MODEL_HORIZONTAL.20.csv is read
TEST_MODEL_HORIZONTAL.21_RESULT.csv is created
TEST_MODEL_HORIZONTAL.21.csv is read
TEST_MODEL_HORIZONTAL.22_RESULT.csv is created
TEST_MODEL_HORIZONTAL.22.csv is read
TEST_MODEL_HORIZONTAL.30_RESULT.csv is created
TEST_MODEL_HORIZONTAL.30.csv is read
TEST_MODEL_HORIZONTAL.31_RESULT.csv is created
TEST_MODEL_HORIZONTAL.31.csv is read
TEST_MODEL_HORIZONTAL.32_RESULT.csv is created
TEST_MODEL_HORIZONTAL.32.csv 

# Сравним результаты

In [11]:
import math as m
# Вариант №1
# Батлер - приток к протяженной трещине при псевдоустановившемся режиме 146 с.
def bat_1(lx, dP, permx, mu, permz, rw, h, L, B):
    pi = 3.14
    Rк = lx/2 # м, радиус контура питания
    dP = dP/10*10**6 # Па, перепад давления
    kh = permx/10**15 # м2, проницаемость (в числовой модели изотропный пласт)
    mu = mu/1000 # Па*с, вязкость
    a = lx # м, сторона модели
    kv = permz/10**15
    Rm = 0.5*rw*(1+m.sqrt(kh/kv))
    hm = h*m.sqrt(kh/kv) # lx
    Q5 = (6*kh*L*h*dP)/(mu*lx/2)*86400/B
    return Q5

In [12]:
# метод для добавления данных на график
import pandas as pd 
import plotly.graph_objects as go
from plotly.offline import iplot
from ecl2df import pvt, EclFiles
import numpy as np
import scipy.interpolate

def figure_plot(filename, rw, kh, h, kk, L, zw, re, lx, ly, S, start_date, title):
    # Получаем забойное и пластовое давление на все иттерации
    start_date = pd.to_datetime(start_date)
    df_time = pd.read_csv(f'csv_folder/{filename}.csv', parse_dates=[0], index_col=[0], usecols=['Time' ,'FPR', 'WBHP:P1', 'WOPR:P1'])
    
    fig = go.Figure()
    trace_name = ['OPM', 'Батлер - вертикальная трещина (146 с.)']
    df_y = []
    df_y.append(df_time['WOPR:P1'])
    y_list = []
    for i in df_time.values:
        dP = i[1] - i[2]
        eclfiles = EclFiles(f'model_folder/{filename}')
        dframe = pvt.df(eclfiles)
        indic = dframe['KEYWORD'] == 'PVDO'
        res_df = dframe.loc[indic][['PRESSURE', 'VOLUMEFACTOR', 'VISCOSITY']]
        x = res_df['PRESSURE'].values
        y1 = res_df['VOLUMEFACTOR'].values
        z = np.poly1d(np.polyfit(x, y1, 2))
        xp = np.linspace(0, 600, 100)
        sol_list_x = [i[1], i[1]+0.1, i[1]+0.2, i[1]+0.3]
        sol_list_y = [0, 1, 2, 3]
        interp1 = scipy.interpolate.InterpolatedUnivariateSpline(xp, z(xp))
        interp2 = scipy.interpolate.InterpolatedUnivariateSpline(sol_list_x, sol_list_y)
        new_x = np.linspace(np.array(xp).min(), np.array(xp).max(), 100)
        new_y11 = interp1(new_x)
        new_y21 = interp2(new_x)
        idx1 = np.argwhere(np.diff(np.sign(new_y11 - new_y21)) != 0)
        y_sol = round(new_y11[idx1[0]][0].astype(float),2)
        x_sol = round(new_x[idx1[0]][0].astype(float),2)
        y2 = res_df['VISCOSITY'].values
        z = np.poly1d(np.polyfit(x, y2, 2))
        xp = np.linspace(0, 600, 100)
        sol_list_x = [i[1], i[1]+0.1, i[1]+0.2, i[1]+0.3]
        sol_list_y = [0, 1, 2, 3]
        interp1 = scipy.interpolate.InterpolatedUnivariateSpline(xp, z(xp))
        interp2 = scipy.interpolate.InterpolatedUnivariateSpline(sol_list_x, sol_list_y)
        new_x = np.linspace(np.array(xp).min(), np.array(xp).max(), 100)
        new_y12 = interp1(new_x)
        new_y22 = interp2(new_x)
        idx2 = np.argwhere(np.diff(np.sign(new_y12 - new_y22)) != 0)
        y_sol = round(new_y12[idx2[0]][0].astype(float),2)
        x_sol = round(new_x[idx2[0]][0].astype(float),2)
        y_list.append(bat_1(lx, dP, kh, new_y12[idx2[0]][0], kh*kk, rw, h, L, new_y11[idx1[0]][0]))
    df_y.append(y_list)
    for i in range(0, 2):
        fig.add_trace(go.Scatter(
                x=df_time.index,
                y=df_y[i],
                mode='lines',
                name=trace_name[i]))
    fig.update_xaxes(tickformat='%d.%m.%y')
    fig.update_layout(title=go.layout.Title(text=title),
                           xaxis_title='Дата',
                           yaxis_title='Qж, м3/сут')
    return fig

In [13]:
for i in range(0, 4):
    filename = f'TEST_MODEL_HORIZONTAL.{i}0'
    rw_ = rw[0] # радиус скважины, м
    kh = permx_list[0] # горизонтальная проницаемость, мд
    h = nz*dz # мощность пласта, м
    L = (y_stop[0]-prod_ys[0])*cells_v # длина горизонтального учатска, м
    zw = h-prod_z2s[0] # расстояние от подошвы пласта до ствола, м
    re = lx/2 # радиус контура питания, м
    figure = figure_plot(filename, rw_, kh, h, kk[i], L, zw, re, lx, ly, skin[0], start_date, model_list[0])

    iplot(figure)

In [8]:
for i in range(0, 4):
    filename = f'TEST_MODEL_HORIZONTAL.{i}1'
    rw_ = rw[0] # радиус скважины, м
    kh = permx_list[1] # горизонтальная проницаемость, мд
    h = nz*dz # мощность пласта, м
    L = (y_stop[0]-prod_ys[0])*cells_v # длина горизонтального учатска, м
    zw = h-prod_z2s[0] # расстояние от подошвы пласта до ствола, м
    re = lx/2 # радиус контура питания, м
    figure = figure_plot(filename, rw_, kh, h, kk[i], L, zw, re, lx, ly, skin[0], start_date, model_list[1])

    iplot(figure)

In [9]:
for i in range(0, 4):
    filename = f'TEST_MODEL_HORIZONTAL.{i}2'
    rw_ = rw[0] # радиус скважины, м
    kh = permx_list[2] # горизонтальная проницаемость, мд
    h = nz*dz # мощность пласта, м
    L = (y_stop[0]-prod_ys[0])*cells_v # длина горизонтального учатска, м
    zw = h-prod_z2s[0] # расстояние от подошвы пласта до ствола, м
    re = lx/2 # радиус контура питания, м
    figure = figure_plot(filename, rw_, kh, h, kk[i], L, zw, re, lx, ly, skin[0], start_date, model_list[2])

    iplot(figure)

In [10]:
title = 'Динамика пластового давления по годам'
x_axis = 'Дата'
y_axis = 'Pпл, атм'
model.summ_plot(['FPR'], x_axis=x_axis, y_axis=y_axis, title=title, name=model_list)